In [8]:
import os
from dotenv import load_dotenv
from pprint import pprint
load_dotenv()

True

#### Creating a sample retriever
* Create a retriever
* load data
* create a straight LLM 
* Ask questions

* Load document

In [6]:
from langchain_community.document_loaders import PyPDFLoader
loader= PyPDFLoader("/Users/vikaslakka/Desktop/FSDS/GenAI/poc/scrum_master/ag_assist/dataset/rm_story.pdf")
pages = loader.load_and_split()

In [16]:


pprint(pages[0])

Document(metadata={'source': '/Users/vikaslakka/Desktop/FSDS/GenAI/poc/scrum_master/ag_assist/dataset/rm_story.pdf', 'page': 0}, page_content='Story Details Page 1\nRun By : System Administrator 2024-08-01 01:38:27 Pacific Daylight TimeReport Title: Story Details\nRun Date and Time: 2024-08-01 01:38:27 Pacific Daylight Time\nRun by: System Administrator\nTable name: rm_story\nStory\nNumber: STRY0010046\nTheme:\nEpic:\nState: Complete\nPoints: 1\nSprint: ServiceNow Implementation R1 1Product:\nRelease: HR ServiceNow Implementation \nRelease 1\nAssignment group: ServiceNow HR Development\nAssigned to: Aileen Mottern (Product Owner)\nBlocked: false\nBlocked reason:\nShort description:\nReactivate HR Case modules\nDescription:\nReactivate the plugin in the instance\nAcceptance criteria:\nShuould be able to see HR demo data in HR case table\xa0 \nNotes\nWork notes:\n2024-08-01 01:37:41 - System Administrator (Work notes) \n@Abel Tuter (architect):  Please confirm on the story \n \nRelated R

### We got the pages
* Now we will split the text
* split docs
* create retriever

In [34]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

##3 Split ttext to chunks
documents= RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    
).split_documents(pages)

In [72]:
vector= FAISS.from_documents(documents, OpenAIEmbeddings())
retriever= vector.as_retriever()


In [38]:
retriever.invoke("What is the number of the story?")

[Document(metadata={'source': '/Users/vikaslakka/Desktop/FSDS/GenAI/poc/scrum_master/ag_assist/dataset/rm_story.pdf', 'page': 0}, page_content='Story Details Page 1\nRun By : System Administrator 2024-08-01 01:38:27 Pacific Daylight TimeReport Title: Story Details\nRun Date and Time: 2024-08-01 01:38:27 Pacific Daylight Time\nRun by: System Administrator\nTable name: rm_story\nStory\nNumber: STRY0010046\nTheme:\nEpic:\nState: Complete\nPoints: 1\nSprint: ServiceNow Implementation R1 1Product:\nRelease: HR ServiceNow Implementation \nRelease 1\nAssignment group: ServiceNow HR Development\nAssigned to: Aileen Mottern (Product Owner)\nBlocked: false\nBlocked reason:\nShort description:\nReactivate HR Case modules\nDescription:\nReactivate the plugin in the instance\nAcceptance criteria:\nShuould be able to see HR demo data in HR case table\xa0 \nNotes\nWork notes:\n2024-08-01 01:37:41 - System Administrator (Work notes) \n@Abel Tuter (architect):  Please confirm on the story \n \nRelated 

### We have retriever
* We will create retriever tool that converts retriever to a tool

In [39]:
from langchain.tools.retriever import create_retriever_tool

In [40]:
retriever_tool= create_retriever_tool(
    retriever,
    "story_retrieval",
    "provides information about story retrieval in Agile project"
)

In [73]:
## form a tool
tools=[retriever_tool]

### Create agent
* Get LLM
* attach tools to llm along with prompt

In [41]:
from langchain_openai import ChatOpenAI
llm= ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

* Prompt loading

In [83]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
template= """
You are a helpful assistant. This is about retrieving of story details in Agile project.
            Data provided is a story detail. Kindly respond if you find details of the story for the question asked
            otherwise respond politely that you do not know in your tone."""

story_reader_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### Create Agent now

In [84]:
from langchain.agents import create_tool_calling_agent
agent= create_tool_calling_agent(llm,tools, story_reader_prompt)

### Create Agent executor

In [85]:
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent= agent, tools= tools, verbose=True)

### Lets invoke

In [86]:
agent_executor.invoke({"input": "Hello"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hello', 'output': 'Hello! How can I assist you today?'}

In [87]:
agent_executor.invoke({"input": "What is the working notes of the story?"})



> Entering new AgentExecutor chain...

Invoking: `story_retrieval` with `{'query': 'working notes'}`


Story Details Page 1
Run By : System Administrator 2024-08-01 01:38:27 Pacific Daylight TimeReport Title: Story Details
Run Date and Time: 2024-08-01 01:38:27 Pacific Daylight Time
Run by: System Administrator
Table name: rm_story
Story
Number: STRY0010046
Theme:
Epic:
State: Complete
Points: 1
Sprint: ServiceNow Implementation R1 1Product:
Release: HR ServiceNow Implementation 
Release 1
Assignment group: ServiceNow HR Development
Assigned to: Aileen Mottern (Product Owner)
Blocked: false
Blocked reason:
Short description:
Reactivate HR Case modules
Description:
Reactivate the plugin in the instance
Acceptance criteria:
Shuould be able to see HR demo data in HR case table  
Notes
Work notes:
2024-08-01 01:37:41 - System Administrator (Work notes) 
@Abel Tuter (architect):  Please confirm on the story 
 
Related Records
Defect:
Enhancement:The working notes of the story are as follo

{'input': 'What is the working notes of the story?',
 'output': 'The working notes of the story are as follows:\n- 2024-08-01 01:37:41 - System Administrator (Work notes)\n- @Abel Tuter (architect): Please confirm on the story'}

In [88]:
response= agent_executor.invoke({"input": "What is the assignment group of the story?"})



> Entering new AgentExecutor chain...

Invoking: `story_retrieval` with `{'query': 'assignment group'}`


Story Details Page 1
Run By : System Administrator 2024-08-01 01:38:27 Pacific Daylight TimeReport Title: Story Details
Run Date and Time: 2024-08-01 01:38:27 Pacific Daylight Time
Run by: System Administrator
Table name: rm_story
Story
Number: STRY0010046
Theme:
Epic:
State: Complete
Points: 1
Sprint: ServiceNow Implementation R1 1Product:
Release: HR ServiceNow Implementation 
Release 1
Assignment group: ServiceNow HR Development
Assigned to: Aileen Mottern (Product Owner)
Blocked: false
Blocked reason:
Short description:
Reactivate HR Case modules
Description:
Reactivate the plugin in the instance
Acceptance criteria:
Shuould be able to see HR demo data in HR case table  
Notes
Work notes:
2024-08-01 01:37:41 - System Administrator (Work notes) 
@Abel Tuter (architect):  Please confirm on the story 
 
Related Records
Defect:
Enhancement:The assignment group of the story is "Se

In [89]:
response['output']

'The assignment group of the story is "ServiceNow HR Development".'

#### Lets Stick to only result now
* disable verbose

In [90]:
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent= agent, tools= tools, verbose=False)

In [91]:
response= agent_executor.invoke({"input": "defects in the story?"})
response['output']

'The story does not mention any defects.'

In [92]:
response= agent_executor.invoke({"input": "Does story has any enhancements in the story?"})
response['output']

'The story does not have any enhancements mentioned in the details.'

In [93]:
response= agent_executor.invoke({"input": "What is working story of the story?"})
response['output']

'The working story of the story is to reactivate HR Case modules by reactivating the plugin in the instance. The acceptance criteria include being able to see HR demo data in the HR case table.'

In [94]:
response= agent_executor.invoke({"input": "What is working notes of the story?"})
response['output']

'The working notes of the story are as follows:\n- 2024-08-01 01:37:41 - System Administrator (Work notes)\n  @Abel Tuter (architect): Please confirm on the story'

In [95]:
response= agent_executor.invoke({"input": "To Whom this story set reviewer?"})
response['output']

'The story is set to be reviewed by Aileen Mottern, who is the Product Owner.'

In [96]:
response= agent_executor.invoke({"input": "What is the summary of the ticket?"})
response['output']

'The summary of the ticket is "Reactivate HR Case modules"'